<a href="https://colab.research.google.com/github/ChristianFML/HDT11-FLOYD/blob/master/DS6011_HT02_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hoja de Trabajo \# 2-1 


---


por Josué Obregón <br>
DS6011 - Feature Engineering <br>
UVG Masters - Escuela de Negocios<br>


## Objetivos

El objetivo de esta hoja de trabajo  es presentar al estudiante diferentes técnicas de visualización de datos para ayudar a entender las características y el comportamiento de la variable objetivo en los modelos predictivos.

Así mismo, las técnicas también son útiles para entender la relación de las distintas variables predictoras, entre ellas mismas y con la variable objetivo.

Como objetivo adicional, también se busca que el estudiante conozca el funcionamiento de dos librerías de visualización de datos del lenguaje python: [Seaborn](https://seaborn.pydata.org/) y [Plotly express](https://plotly.com/python/plotly-express/).

El análisis presentado en esta hoja de trabajo está basado en la [sección 4.2](http://www.feat.engineering/visualizations-for-numeric-data-exploring-train-ridership-data.html) del libro **Feature engineering and selection: A practical approach for predictive models**. Intentaremos generar las visualizaciones lo más fielmente posible a las presentadas en el libro.


## Importación de librerías y carga de los datos a varios pandas [DataFrames](https://pandas.pydata.org/pandas-docs/version/1.1.5/reference/frame.html)


Las librerías que importaremos para empezar son pandas y numpy para el manejo de los datos, y matplotlib, seaborn y plotly para la generación de visualizaciones. 

Los datos fueron obtenidos del [portal de datos de Chicago](https://data.cityofchicago.org/Transportation/CTA-Ridership-L-Station-Entries-Daily-Totals/5neh-572f).

Los datos necesitan cierto preprocesamiento, pero el objetivo de esta hoja de trabajo no es ese, así que incluyo dos archivos extra. El primero es donde están registrados los días usados para el análisis, y el segundo son atributos creados por el autor del libro. 



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
!mkdir data

In [ ]:
import gdown

urls = ['https://drive.google.com/uc?export=download&id=1GiEi6LVHBAAX4IWVA2gRH1gJZ1vLYghG', # chicago_train_data  https://drive.google.com/file/d/1GiEi6LVHBAAX4IWVA2gRH1gJZ1vLYghG/view?usp=sharing
        'https://drive.google.com/uc?export=download&id=1LIK0YFER5ve9Wn7Uap4Zb6f5pXcAExZ-', # train_days   https://drive.google.com/file/d/1LIK0YFER5ve9Wn7Uap4Zb6f5pXcAExZ-/view?usp=sharing 
        'https://drive.google.com/uc?export=download&id=18pIOZQXBOSwT-t69icoGZgZh-kUUFMrN' # extra data chicago https://drive.google.com/file/d/18pIOZQXBOSwT-t69icoGZgZh-kUUFMrN/view?usp=sharing
        ]
outputs = ['chicago_train_data.csv','train_days.csv' ,'extra_data_chicago.csv']
for url,output in zip(urls,outputs):
  gdown.download(url, f'data/{output}', quiet=False)

In [ ]:
df = pd.read_csv('data/chicago_train_data.csv', parse_dates=True, index_col=2)
train_days = pd.read_csv('data/train_days.csv',index_col=1, parse_dates=True)
extra_data = pd.read_csv('data/extra_data_chicago.csv',index_col=0, parse_dates=True)

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
df = df.sort_index()
df.info()

In [ ]:
df.loc[train_days.index]

In [ ]:
df.index.year.unique()

In [ ]:
df_cl = df[df.stationname=='Clark/Lake'][['daytype'	,'rides']] # let's focus on Clark/Lake station
df_cl ['rides_x1000']= df_cl.rides/1000 # transform 
df_cl['part_of_the_week']=df_cl.index.weekday.map(lambda x: 'Weekday' if x<5 else 'Weekend')

In [ ]:
df_cl.index.duplicated().sum()

In [ ]:
df_cl = df_cl[~df_cl.index.duplicated(keep='first')] # there was one record duplicated index 3815 value '2011-07-03'
print(df_cl.index.duplicated().sum())
train_df_cl = df_cl.loc[train_days.index] # define our training data with train days

## Gráficos de cajas, Gráficos de violin, Histogramas y Gráficos de densidad



In [ ]:
sns.set_style("darkgrid") # Others whitegrid, white, dark, ticks https://www.python-graph-gallery.com/104-seaborn-themes 

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(12,8)) #create grid
sns.boxplot(data=train_df_cl, x='rides_x1000', orient='h', ax=axs[0,0])
sns.histplot(data=train_df_cl,x='rides_x1000', binwidth=0.7, ax=axs[0,1])
sns.violinplot(data=train_df_cl,x='rides_x1000', ax=axs[1,0])
sns.kdeplot(data=train_df_cl, x='rides_x1000', ax=axs[1,1])
plt.tight_layout()

### Aumentando las visualizaciones con facetas, colores y formas

In [ ]:
g = sns.FacetGrid(train_df_cl, sharey=False, row='part_of_the_week',  hue='part_of_the_week',height=3, aspect=3)
g.map_dataframe(sns.histplot, x='rides', log_scale=True, binwidth=0.03)

## Gráficos de dispersión



In [ ]:
train_df_cl['two_week_lag_rides'] = train_df_cl['rides'].shift(periods=14, fill_value = 0) # We create a lagging variable of two weeks
train_df_cl ['two_week_lag_rides_x1000']= train_df_cl['two_week_lag_rides']/1000

In [ ]:
train_df_cl.head(16)

In [ ]:
plt.figure(figsize=(6,6)) 
sns.scatterplot(data=train_df_cl, x='two_week_lag_rides_x1000', y='rides_x1000', hue='part_of_the_week')

## Mapas de calor

Creamos una variable indicador para valores entre semana que sean menores a 10,000 pasajeros

In [ ]:
train_df_cl['less_than_10000']= train_df_cl.rides<=10000
train_df_cl['less_than_10000'] = train_df_cl['less_than_10000'].astype(int) # to transforme it to int, i.e. True=1 and False=0
train_df_cl['month_name'] = train_df_cl.index.month_name() 
train_df_cl['month'] = train_df_cl.index.month
train_df_cl['day'] = train_df_cl.index.day
train_df_cl['month_day']= train_df_cl['month'].apply('{:0>2d}'.format) +'-' + train_df_cl['day'].apply('{:0>2d}'.format) # To create a format mm-dd
train_df_cl['year'] = train_df_cl.index.year

In [ ]:
train_df_cl['2001-02'].head()

Para crear la tabla fuente de nuestro mapa de calor, utilizamos la [función de tabla pivote de Pandas](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html).

In [ ]:
heat_map_data = train_df_cl[train_df_cl.part_of_the_week=='Weekday'].pivot_table(index='month_day', columns='year', values='less_than_10000', fill_value=0) 

In [ ]:
heat_map_data.head(10)

In [ ]:
plt.figure(figsize=(10,10))
ax_hm = sns.heatmap(data=heat_map_data, cmap=['white','red'], cbar=False, yticklabels=15)
ax_hm.invert_yaxis()

Ahora que conocemos que el efecto de los días festivos en el conjunto de datos, podemos remover esos días.

Para eso utilizamos el dataset de **extra\_data** que cargamos al principio.

In [ ]:
common_holidays = ['USNewYearsDay', 'Jan02_Mon_Fri', 'USMLKingsBirthday', 
    'USPresidentsDay', 'USMemorialDay', 'USIndependenceDay', 
    'Jul03_Mon_Fri', 'Jul05_Mon_Fri', 'USLaborDay', 'USThanksgivingDay', 
    'Day_after_Thx', 'ChristmasEve', 'USChristmasDay', 'Dec26_wkday', 
    'Dec31_Mon_Fri']

In [ ]:
dates_with_holiday = (extra_data[common_holidays]==1).any(axis=1)

In [ ]:
train_df_cl['holiday']=dates_with_holiday

In [ ]:
train_df_cl['two_weeks_lag_holiday'] = train_df_cl['holiday'].shift(periods=14, fill_value = False)

Filtramos el conjunto de datos con dias que no sean festivos y que no tengan un dia festivo con retardo de dos semanas.

In [ ]:
train_df_cl_no_holiday= train_df_cl[~train_df_cl.holiday & ~train_df_cl.two_weeks_lag_holiday]

In [ ]:
plt.figure(figsize=(6,6))
sns.scatterplot(x=train_df_cl_no_holiday.two_week_lag_rides_x1000, y=train_df_cl_no_holiday.rides_x1000, hue=train_df_cl_no_holiday.part_of_the_week)

## Gráficos de matriz de correlación

Agregamos las características que creamos para nuestro subconjunto de datos conteniendo solo la estacion Clark/Lake.

Despues de esto, tomamos solo los registros correspondientes al año 2016

In [ ]:
df['part_of_the_week']=df.index.weekday.map(lambda x: 'Weekday' if x<5 else 'Weekend')
df['holiday'] = False
df['two_week_lag_rides']=0
for station in df.stationname.unique():
  df.loc[df.stationname==station,'two_week_lag_rides'] = df[df.stationname==station]['rides'].shift(periods=14, fill_value = 0)
for index, row in dates_with_holiday.iteritems():
  if row:        
    df.loc[index, 'holiday']=True


all_2016 = df.loc[train_days.index]['2016'].copy()

In [ ]:
all_2016.head()

Tomamos los datos solo de días entre semana y que no son días festivos

In [ ]:
clustermap_data = all_2016[(all_2016.part_of_the_week=='Weekday')&(~all_2016.holiday)][['stationname','two_week_lag_rides']]

In [ ]:
clustermap_data.head()

Creamos la tabla pivote donde las filas son los días del año 2016, y las columnas son las estaciones. Cada celda en el conjunto de datos representa la cantidad de pasajeros para una estación y día específicos.

In [ ]:
clustermap_data.pivot(columns='stationname', values='two_week_lag_rides')

Creamos la matriz de correlación, y luego generamos la gráfica. La cual también incluye una función de clusterización para las estaciones. Es decir, las estaciones con correlación positiva mayor se encuentran contiguas y las estaciones con correlación negativa mayor se encuentran alejadas. Esto es representado con un dendograma en los ejes de la gráfica.

In [ ]:
corr_matrix = clustermap_data.pivot(columns='stationname', values='two_week_lag_rides').corr()

In [ ]:
corr_matrix.loc['O\'Hare Airport','UIC-Halsted']

In [ ]:
corr_matrix.loc['Harlem-Lake','Quincy/Wells']

In [ ]:
clustermap = sns.clustermap(corr_matrix, cmap='RdBu_r', vmin=-1, vmax=1, figsize=(10 ,10))


## Gráficos de líneas

In [ ]:
from scipy.stats.mstats import gmean

In [ ]:
line_plot_data_gmean = train_df_cl.groupby([pd.Grouper(freq='M'),'part_of_the_week']).agg({'rides_x1000':gmean, 'month_name':max, 'year':max}).reset_index()
line_plot_data_gmean.set_index('date', inplace=True)
line_plot_data_mean = train_df_cl.groupby([pd.Grouper(freq='M'),'part_of_the_week']).agg({'rides_x1000':'mean', 'month_name':max, 'year':max}).reset_index()
line_plot_data_mean.set_index('date', inplace=True)

In [ ]:
line_plot_data_gmean.head()

In [ ]:
gline = sns.FacetGrid(line_plot_data_gmean, sharex=False, col='part_of_the_week', hue='year',height=6, aspect=2, palette='gist_heat_r')
gline.map_dataframe(sns.lineplot, x='month_name', y='rides_x1000', legend='full')
gline.axes[0,1].legend()
gline.set_xticklabels(rotation=30)

In [ ]:
gline = sns.FacetGrid(line_plot_data_mean, sharex=False, col='part_of_the_week', hue='year',height=6, aspect=2, palette='gist_heat_r')
gline.map_dataframe(sns.lineplot, x='month_name', y='rides_x1000', legend='full')
gline.axes[0,1].legend()

In [ ]:
fig = px.line(line_plot_data_gmean, x='month_name', y='rides_x1000', facet_col='part_of_the_week', color='year', color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.update_xaxes(tickangle=-45)
fig.show()

Los precios promedio por semana en Chicago se han registrado. Cargamos el archivo y generemos un gráfico de líneas para observar si hay alguna relación con el número de pasajeros.

In [ ]:
gas_price_df = pd.read_csv('/content/drive/MyDrive/UVG-DS6011/session_02/data/Chicago_gas_prices.csv',index_col=0, parse_dates=True)

In [ ]:
gas_price_df = gas_price_df.sort_index()

In [ ]:
gas_price_df.head()

In [ ]:
gas_price_df['month_name'] = gas_price_df.index.month_name() 
gas_price_df['year'] = gas_price_df.index.year 
gas_price_df['part_of_the_week']=gas_price_df.index.weekday.map(lambda x: 'Weekday' if x<5 else 'Weekend')

In [ ]:
gas_price_df.head()

In [ ]:
line_plot_gas = gas_price_df['2001-01':'2016-08'].groupby(pd.Grouper(freq='M')).agg({'gas_price':'mean', 'month_name':max, 'year':max}).reset_index()
line_plot_gas.set_index('date', inplace=True)

In [ ]:
line_plot_gas

In [ ]:
fig = px.line(line_plot_gas, x='month_name', y='gas_price', color='year', color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.show()

Ahora veamos si hay alguna relación entre los precios promedio de la gasolina y el número de pasajeros. 

Para esto calculamos el promedio mensual de los precios de la gasolina con un retardo de 2 semanas y lo graficamos en contra de la media geométrica de el número de pasajeros en la estación de Clark/Lake

In [ ]:
gas_price_df['two_week_lag_gas_price'] = gas_price_df.gas_price.shift(periods=14,freq='d')
gas_price_df['two_week_lag_gas_price'] = gas_price_df['two_week_lag_gas_price'].fillna(0)

In [ ]:
gas_price_df.head()

In [ ]:
line_plot_gas_lagged = gas_price_df['2001-01-01':'2016-08-31'].groupby(pd.Grouper(freq='M')).agg({'gas_price':'mean', 'month_name':max, 'year':max}).reset_index()
line_plot_gas_lagged.set_index('date', inplace=True)

In [ ]:
line_plot_gas_lagged.head()

In [ ]:
line_plot_data_gmean.loc[line_plot_data_gmean.part_of_the_week=='Weekday', 'gas_price'] = line_plot_gas_lagged
line_plot_data_gmean.loc[line_plot_data_gmean.part_of_the_week=='Weekend', 'gas_price'] = line_plot_gas_lagged

In [ ]:
line_plot_data_gmean.head()

In [ ]:
line_plot_data_gmean.year = line_plot_data_gmean.year.astype(str) # we make it string so it is considered categorical variable

In [ ]:
fig = px.scatter(line_plot_data_gmean, x='gas_price', y='rides_x1000', color='year', facet_col='part_of_the_week', color_discrete_sequence=px.colors.qualitative.Alphabet) #, trendline='ols')
fig.show()

# PCA

In [ ]:
lag_14_data = pd.read_csv('/content/drive/MyDrive/UVG-DS6011/session_02/data/lag_14_data.csv', index_col=0)

In [ ]:
lag_14_data.head()

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA()
pca.fit(lag_14_data)

In [ ]:
print(pca.n_components_)
print(pca.components_.shape) #eigenvectors

In [ ]:
pca.singular_values_ #eigenvalues

In [ ]:
projected_lag_14 = pca.transform(lag_14_data)
projected_lag_14 = projected_lag_14*-1 # just for matching the book's result https://stackoverflow.com/questions/44765682/in-sklearn-decomposition-pca-why-are-components-negative

In [ ]:
len(pca.components_)

In [ ]:
variance_ration_cumsum = np.cumsum(np.pad(pca.explained_variance_ratio_, (1, 0), "constant"))

In [ ]:
fig =px.line(x=range(50), y=variance_ration_cumsum[:50], width=500, height=500)
fig.update_yaxes(range=[-0.02, 1.02])
fig.update_xaxes(range=[-2, 53])

In [ ]:
fig =px.scatter(x=projected_lag_14[:,0], y = projected_lag_14[:,1], color_discrete_sequence=['black'], width=500, height=500, opacity=0.3)
fig.update_yaxes(range=[-50, 45])
fig.update_xaxes(range=[-50, 45])
fig.show()

In [ ]:
projected_df = pd.DataFrame(projected_lag_14[:, :2], columns=['Component1','Component2'])

In [ ]:
projected_df.index = train_df_cl.index
projected_df['day'] = projected_df.index.day_name()
projected_df['year'] = projected_df.index.year

In [ ]:
fig = px.violin(projected_df,x='day',y='Component1',width=1000, height=400)
fig.layout.template = 'plotly_white' #  different template styles https://plotly.com/python/templates/ 
fig.show()

In [ ]:
fig = px.violin(projected_df,x='year',y='Component2',width=1000, height=400)
fig.layout.template = 'plotly_dark' 
fig.show()